In [ ]:
import numpy as np

In [ ]:
[2,2,1][:1]
layers=[2,2,1]
layers = [print(n_neurons,n_inputs) for n_neurons, n_inputs in zip(layers, layers[:-1])]

[2]

In [ ]:
class ActivationFunction:
    def __init__(self, func, derivative, name):
        self.func = func
        self.derivative = derivative
        self.name = name

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

sigmoid_activation = ActivationFunction(sigmoid, sigmoid_derivative, 'sigmoid')



In [ ]:
class NeuralNetwork:
    def __init__(self, layers, activation_function):
        self.layers = [Layer(n_neurons, activation_function, n_inputs) for n_neurons, n_inputs in zip(layers, layers[:-1])]
        # self.layers = [Layer(n_neurons, activation_function) for n_neurons in layers]
        self.activation_function = activation_function

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    # def backward(self, inputs, expected_output, learning_rate):
    #     # propagate the error through the layers in reverse order
    #     for i in range(len(self.layers) - 1, -1, -1):
    #         layer = self.layers[i]
    #         inputs = layer.backward(inputs, expected_output, learning_rate, self.activation_function)
    
    def backward(self, inputs, expected_output, learning_rate):
        error = expected_output - self.forward(inputs)
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            error = layer.backward(error, learning_rate, self.activation_function)

    def train(self, X, y, learning_rate, epochs):
        for i in range(epochs):
            for j, inputs in enumerate(X):
                output = self.forward(inputs)
                self.backward(inputs, y[j], learning_rate)

In [ ]:
class Layer:
    def __init__(self, n_neurons, activation_function, n_inputs):
        self.neurons = [Neuron(activation_function, n_inputs) for _ in range(n_neurons)]

    def forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.forward(inputs))
        return outputs

    def backward(self, inputs, expected_output, learning_rate, activation_function):
        print(expected_output)
        print(expected_output.shape)
        print(type(expected_output))
        errors = []
        expected_output = np.reshape(expected_output, (len(expected_output), 1))
        print(expected_output)
        print(expected_output.shape)
        print(type(expected_output))
        for i, neuron in enumerate(self.neurons):
            error = expected_output[i] - neuron.output
            errors.append(error)
            neuron.backward(error, learning_rate, activation_function,inputs)



In [ ]:
class Neuron:
    def __init__(self, activation_function, n_inputs):
        self.weights = np.random.randn(n_inputs)
        self.bias = np.random.randn()
        self.output = None
        self.activation_function = activation_function

    def forward(self, inputs):
        weighted_sum = np.dot(inputs, self.weights) + self.bias
        self.output = self.activation_function.func(weighted_sum)
        return self.output

    def backward(self, error, learning_rate, activation_function,inputs):
        # update the weights and bias
        derivative = activation_function.derivative(self.output)
        self.weights += learning_rate * error * derivative * inputs
        self.bias += learning_rate * error * derivative


In [ ]:

# # create the neural network with 2 input neurons, 2 hidden neurons, and 1 output neuron
# nn = NeuralNetwork([2, 2, 1], sigmoid_activation)

# # create the dataset with input and output values
# X = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])
# y = np.array([[1], [0], [0], [0]])

# # train the network
# nn.train(X, y, learning_rate=0.1, epochs=1000)

# # make a prediction for the first input in the dataset
# inputs = X[0]
# output = nn.forward(inputs)
# print(output) # [0.5]


In [ ]:
import numpy as np

class Neuron:
    def __init__(self, n_inputs, activation_function):
        self.weights = np.random.random(n_inputs)
        self.bias = np.random.randn()
        self.output = None
        self.activation_function = activation_function

    def forward(self, inputs):
        z = np.dot( self.weights,inputs) + self.bias
        self.output = self.activation_function(z)
        return self.output

    def backward(self, error, learning_rate, activation_function_derivative,inputs):
        delta = error * activation_function_derivative(self.output)
        self.weights -= learning_rate * delta * inputs
        self.bias -= learning_rate * delta

class Layer:
    def __init__(self, n_neurons, n_inputs, activation_function):
        self.neurons = [Neuron(n_inputs, activation_function) for _ in range(n_neurons)]
        self.output = None

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.array([neuron.forward(inputs) for neuron in self.neurons])
        return self.output

    def backward(self, error, learning_rate, activation_function, activation_function_derivative):
        errors = []
        error = np.dot(error, np.transpose(self.weights))
        for i, neuron in enumerate(self.neurons):
            errors.append(error[i] * activation_function_derivative(neuron.output))
            neuron.backward(error[i], learning_rate, activation_function_derivative, self.inputs)
        return errors

class NeuralNetwork:
    def __init__(self, architecture, activation_function, activation_function_derivative):
        self.layers = [Layer(n_neurons, n_inputs, activation_function) for n_neurons, n_inputs in zip(architecture[:-1], architecture[1:])]
        self.output_layer = Layer(architecture[-1], architecture[-2], activation_function)
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return self.output_layer.forward(inputs)

    def backward(self, inputs, expected_output, learning_rate):
        error = expected_output - self.forward(inputs)
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            error = layer.backward(error, learning_rate, self.activation_function, self.activation_function_derivative)
            self.output_layer.backward(error, learning_rate, self.activation_function, self.activation_function_derivative)

    def train(self, inputs, expected_output, learning_rate, epochs):
        for _ in range(epochs):
            for i in range(len(inputs)):
                self.forward(inputs[i])
                self.backward(inputs[i], expected_output[i], learning_rate)

    def predict(self, inputs):
        return self.forward(inputs)


In [ ]:
# create the neural network with 2 input neurons, 2 hidden neurons, and 1 output neuron
nn = NeuralNetwork([2, 2, 1], sigmoid,sigmoid_derivative)

# create the XOR dataset
X = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])
y = np.array([[0], [1], [1], [0]])

# train the network
nn.train(X, y, learning_rate=0.1, epochs=10000)

# make predictions for the XOR dataset
for inputs, expected_output in zip(X, y):
    output = nn.forward(inputs)
    print(f'Inputs: {inputs}, Output: {output}, Expected Output: {expected_output}')


ValueError: ignored